In [1]:
import numpy as np
import autolens as al
import autolens.plot as aplt
import os
%load_ext autoreload

import sys
from pathlib import Path

base_dir = Path.cwd()
autolens_config_path = base_dir / "autolens_config"
spherex_path = base_dir / "SPHEREx"

sys.path.append(str(autolens_config_path))
sys.path.append(str(spherex_path))


#import gcluster15  # from autolens_config
import SPHERExScripts as SPHEREx  # from SPHEREX
import gcluster15 as galaxy
import autolens_config



2025-01-30 15:07:20,399 - numexpr.utils - INFO - Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2025-01-30 15:07:20,400 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
%autoreload 2

## Here are some properties about the galaxy clusters we are studying and simulating I want to note 
***

First of all, we want to look at *rich* galaxy clusers (N_gal >= 30). Rich galaxy clusters are known to be proponents of strong lensing. We can see this from the papers:
- [https://ui.adsabs.harvard.edu/abs/2011RAA....11.1185W/]
- [https://ui.adsabs.harvard.edu/abs/2009RAA.....9....5W/]
- [https://ui.adsabs.harvard.edu/abs/2014SCPMA..57.1809L/]

However, we need some way to parametrize the mass and light given the number of galaxies and redshift. After all, we are creating mass and light profiles so that PyAutoLens can perform raytracing. 

We can use the results from [https://iopscience.iop.org/article/10.1086/444554/pdf]:
* $R^{N}_{200} \sim N^{0.6}_{gal}$, powerlaw related by index $\approx -0.6$

* $M_{200} \sim N^{1.8}_{gal}$ 

* $M^{1/3}_{200} \sim N^{1/3\alpha\beta}_{gal}$ where $(1/3\alpha\beta) \approx 0.6$ and $\beta$ in the range $\sim 0.5 - 0.65$ and $\alpha$ ranges depending on $N_{gal}$, but $\alpha =-1$ for $N_{gal} > 8$
Where $R^{N}_{200}$ is the characteristic radius, $N_{gal}$ is the cluster richness, and the $200$ subscript denotes the virial term. 


Now, we need to model the dark matter halo. Going off what Cooray said, we can have the dark matter halo as about 30" in radius. However, It would be better if we had a better estimate related to the number of galaxies. 

We can describe the dark matter profile of a galaxy using an NFW:
- [https://iopscience.iop.org/article/10.1086/500288/pdf]
- [https://arxiv.org/pdf/astro-ph/9508025]
- [https://iopscience.iop.org/article/10.1088/0004-637X/729/2/127/pdf]
- [https://iopscience.iop.org/article/10.1086/308744/pdf]


We also need to figure out how to distribute the galaxies within the plane of the 2D image. We can use the standard power law formula for the distribution:
- $n(R) \propto (1 + R/R_c)^{-\alpha}$

where $R$ is the cluster centric radius (the distance from the galaxy to the center of the cluster, which in our case will always be the center of the plane) and $R_c$ is the cluster radius. 

---
#### Simulation
Since PyAutoLens does not have formal support for creating galaxy clusters, we have to create each individual galaxy. We can break this down into mass profiles, and light profiles. 

In the case of the mass profile, the NFW dark matter halo will dominate the galaxy cluster's mass distribution. The percentage that it take is approximately 90%.
(https://ui.adsabs.harvard.edu/abs/1996ApJ...462..563N/abstract)
(https://www.mdpi.com/2075-4434/7/1/8)
(https://www.nature.com/articles/s41586-020-2642-9)
Therefore, we can accurately model the mass profile with the dark matter halo NFW. Perhaps we can try adding in a small correction due to some extraneously dense galaxy, but such an implementation is not necessary for now. 


We also need to model the light profile. This one is more tricky, for obvious reasons. 
Naively, we can create a random sersic light profile for every single galaxy in the cluster, add all the galaxies to our galaxy cluster, and we have a light profile for our cluster. However, this is computationally expensive and not exactly accurate. 

Here are some facts we know about galaxy clusters, their member galaxies and their light profiles. 
- They can be described by sersic profiles

- There exists a "brightest" galaxy within the cluster, positioned near the center of the cluster, which can take up to 36% of the total light profile. (https://arxiv.org/pdf/astro-ph/0001415)

- There are a range of luminous galaxy members, but we will proceed by truncating their contributions under a certain apparent magnitude. This is to reduce computaional intensity. We can allow the medium range luminous galaxies to have a light profile within a certain random range. 


Now, we need to get the flux to pixel ratio correct. After all, these are going to be placed into the background of images from the merged catalog of DESI + Subaru legacy surveys. 
From what I can see, this has to be aligned with the pixel scales from marco's images. 



In [3]:
# imageRaw = galaxy.wrapperFunction()

## Citations
- https://ui.adsabs.harvard.edu/abs/2011RAA....11.1185W/
- https://ui.adsabs.harvard.edu/abs/2009RAA.....9....5W/
- https://ui.adsabs.harvard.edu/abs/2014SCPMA..57.1809L/
- https://iopscience.iop.org/article/10.1086/444554/pdf
- https://arxiv.org/pdf/1809.03325
- https://www.frontiersin.org/journals/astronomy-and-space-sciences/articles/10.3389/fspas.2024.1411810/full
- https://arxiv.org/pdf/2111.08721
- https://arxiv.org/pdf/2111.08721
- https://www.aanda.org/articles/aa/full_html/2013/07/aa21268-13/aa21268-13.html
- https://iopscience.iop.org/article/10.1086/340952/pdf
- https://www.researchgate.net/figure/The-Sersic-profile-equation-1-describes-how-a-galaxy-light-profile-varies-as-a-function_fig2_234382030
- https://arxiv.org/pdf/astro-ph/0001415
- https://www.aanda.org/articles/aa/full_html/2021/05/aa40245-20/aa40245-20.html
- 